## Metadata VDB Search
We will use Milvus and Towhee to help searches. Towhee is used to extract the semantics of the text and return the text embedding. The Milvus vector database can store and search vectors, and return related dataset's metadata. So we first need to install [Milvus](https://github.com/milvus-io/milvus) and [Towhee](https://github.com/towhee-io/towhee).

Before getting started, please make sure that you have started a [Milvus service](https://milvus.io/docs/install_standalone-docker.md). This notebook uses [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.2.11](https://milvus.io/docs/release_notes.md#2210).

### Package installations

In [ ]:
#! pip install --upgrade pip
#! pip3 install -q towhee pymilvus==2.2.11
#! pip3 uninstall pymilvus -y

! pip3 install -q towhee pymilvus==2.1.1
! pip3 show pymilvus | grep -Ei 'Name:|Version:'
! pip3 show towhee | grep -Ei 'Name:|Version:'

## 1.1 Adding embeddings for columns

The dataset is from the [Kartverket dataset metadata](https://cdn.discordapp.com/attachments/1204433663035449384/1206537816654356480/metadata_no_format.csv?ex=65dc5ee7&is=65c9e9e7&hm=3b9a88db41103ef5393294c5eaeebb60ee2229f43724cc014d4cffc92de1f384&), which contains metadata about each dataset.

The strings in the columns need to be converted to vector representations (embedding) using Towhee [text_embedding.dpr operator](https://towhee.io/text-embedding/dpr). Columns containing these new embedings should contain the original column name with `_vector` at the end.

### NB In case pandas cannot read the csv, due to a delimiter parsing error

Use the code below to reformat the delimiters to "|", `NB! Also replace the excess ones inside sentences that replaced the regular commas.`

In [ ]:
# Cell for reformatting the delimiters to "|"
import re
import csv

def replace_delimiter(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        content = file.read()

    # Regular expression to match commas not inside double quotes
    pattern = r',(?=(?:[^"]*"[^"]*")*[^"]*$)'

    # Replace the matched commas with '|'
    new_content = re.sub(pattern, '|', content)

    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(new_content)

# Replace this with your actual file paths
input_file = 'metadata.csv'
output_file = 'output_metadata_modified.csv'

replace_delimiter(input_file, output_file)


## 2. Load dataset and vectorise chosen column

In [ ]:
import pandas as pd
from towhee import pipe, ops, DataCollection
from tqdm import tqdm


# Function to compute embeddings for a single text
def compute_embeddings(text):
    MAX_TOKENS = 512 # Temporary limit on number characters to convert
    truncated_text = text[:MAX_TOKENS]
    return DataCollection(embeddings_pipe(truncated_text)).to_list()[0]['vec']


# Loads dataset into dataframe and recasts columns into correct datatypes
df_kartverket = pd.read_excel('Metadata_excel.xlsx')
recast_to_string = ['datasetcreationdate', 'metadatacreationdate']
df_kartverket[recast_to_string] = df_kartverket[recast_to_string].astype('object')

# Fill NaN values with an empty string
df_kartverket.fillna('', inplace=True)

# Pipe converting text to embeddings (vectors)
embeddings_pipe = (
    pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .output('vec')
)

# Process each column and create new columns for embeddings
column_to_vectorise = 'title'
df_kartverket[column_to_vectorise + '_vector'] = df_kartverket[column_to_vectorise].apply(compute_embeddings)

In [ ]:
from towhee import pipe, ops

def insert_data_to_milvus_with_towhee(df, server_host, server_port, collection_name):
    try:
        # Define the pipeline
        insert_pipe = (pipe.input('data_frame')
                       .flat_map('data_frame', 'data', lambda df: df.values.tolist())
                       .map('data', 'res', ops.ann_insert.milvus_client(host=server_host, 
                                                                        port=server_port,
                                                                        collection_name=collection_name))
                       .output('res'))

        # Execute the pipeline
        results = insert_pipe(df)
        return results
    except Exception as e:
        print(f"Error during insertion: {e}")

# Usage
server_host = 'ebjerk.no'
server_port = '19530'
collection_name = 'kartverket_metadata'

inserted_ids = insert_data_to_milvus_with_towhee(df_kartverket, server_host, server_port, collection_name)

## 3. Creation of Milvus collection for metadata

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

server_host = 'ebjerk.no'
server_port = '19530'

connections.connect(host=server_host, port=server_port)

def kartverket_create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
            #FieldSchema(name='schema', dtype=DataType.STRING, max_length=100), # REQUIRES STRING TYPE 
            FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=False),
            #FieldSchema(name='uuid', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE
            #FieldSchema(name='hierarchyLevel', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE   
            #FieldSchema(name='hierarchyLevel_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), #REQUIRES STRING TYPE   
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=100),   
            FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=dim),

            #FieldSchema(name='datasetcreationdate', dtype=DataType.VARCHAR, max_length=500), # REQUIRES STRING TYPE   
            FieldSchema(name='abstract', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='abstract_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='keyword', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='keyword_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            #FieldSchema(name='geoBox', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE   
            #FieldSchema(name='geoBox_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), # REQUIRES STRING TYPE   
            FieldSchema(name='Constraints', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='Constraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            FieldSchema(name='SecurityConstraints', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='SecurityConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='LegalConstraints', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='LegalConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            #FieldSchema(name='temporalExtent', dtype=DataType.VARCHAR, max_length=100), # REQUIRES STRING TYPE   
            ##FieldSchema(name='temporalExtent_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), # REQUIRES STRING TYPE   
            #FieldSchema(name='image', dtype=DataType.VARCHAR, max_length=1000), # REQUIRES STRING TYPE   
            FieldSchema(name='responsibleParty', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='responsibleParty_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            #FieldSchema(name='link', dtype=DataType.VARCHAR, max_length=500), # REQUIRES STRING TYPE   
            #FieldSchema(name='metadatacreationdate', dtype=DataType.VARCHAR, max_length=500), # REQUIRES STRING TYPE   
            ##FieldSchema(name='metadatacreationdate_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), # REQUIRES STRING TYPE   
            FieldSchema(name='productInformation', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='productInformation_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='parentId', dtype=DataType.VARCHAR, max_length=100),   
    ]
    schema = CollectionSchema(fields=fields, description='search text')
    collection = Collection(name=collection_name, schema=schema)
    
    index_params = {
        'metric_type': "L2",
        'index_type': "IVF_FLAT",
        'params': {"nlist": 2048}
    }
    collection.create_index(field_name='title_vector', index_params=index_params)
    return collection

kartverket_collection = kartverket_create_milvus_collection('kartverket_metadata', 768)

### 3.1 Creation of dataframe subset to exclude columns with complex data types

In [ ]:
df_kartverket_slice = df_kartverket[['id', 'title', 'title_vector', 'abstract', 'keyword', 'Constraints', 'SecurityConstraints', 'LegalConstraints', 'responsibleParty', 'productInformation', 'parentId']]
df_kartverket_slice

### 3.2 Insert the subset dataframe data into Milvus collection

In [ ]:
from towhee import ops, pipe, DataCollection

insert_pipe_kartverket = (pipe.input('df_kartverket_slice')
                   .flat_map('df_kartverket_slice', 'data', lambda df: df.values.tolist())
                   .map('data', 'res', ops.ann_insert.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name='kartverket_metadata'))
                   .output('res')
)

%time _ = insert_pipe_kartverket(df_kartverket_slice)


kartverket_collection.load()
kartverket_collection.num_entities

## 4. Query against metadata collection

In [ ]:
import numpy as np
# Variables specifying what column and collection to perform ANN comparrison against
vector_columns = ['title_vector']
collection_name = 'kartverket_metadata'

print(df_kartverket.columns)
# What columns to return for view
response_output = [
       'id', 'title',
       'abstract', 'keyword', 'Constraints',
       'SecurityConstraints', 'LegalConstraints',
       'responsibleParty', 'productInformation', 'parentId'
]


demo_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'title',
       'abstract', 'keyword', 'Constraints',
       'SecurityConstraints', 'LegalConstraints',
       'responsibleParty', 'productInformation', 'parentId'), 
                                       ops.ann_search.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name=collection_name,
                                                                    vector_field=vector_columns,
                                                                    output_fields=response_output, 
                                                                    limit=5))  
                    .output(*['query', 'score'], *response_output)
               )

kartverket_question1 = 'Just do it'
print(f'\n"{kartverket_question1}" search:')
res_kartverket1 = demo_pipe(kartverket_question1)
DataCollection(res_kartverket1).show()